necessary packages

In [75]:
import numpy as np
from ipywidgets import interact
import plotly.graph_objects as go
from scipy.special import gamma
from scipy.integrate import quad as integrate

define function for updating the layout of the figure:
* updates the layout of a plotly figure with predefined settings
* the **title** dict includes all title properties: **'x'** specifies the position of the title along the x-axis, **'xanchor'** specifies the anchor point of the title along the x-axis

In [76]:
def update_layout_of_graph(fig: go.Figure, title: str = 'Plot') -> go.Figure:
    fig.update_layout(
        width=1000,
        height=500,
        autosize=False,
        plot_bgcolor='black',
        title={'text': title, 'x': 0.5, 'xanchor': 'center'},  # dict includes all title properties; specifies the position of the title along the x-axis
        xaxis_title='t / ms',
        yaxis_title='I / A',
        legend=dict(yanchor="top", y=0.4, xanchor="right", x=0.95),
        xaxis=dict(showline=True, linewidth=1, linecolor='black'),
        yaxis=dict(showline=True, linewidth=1, linecolor='black')
    )
    return fig

define function for creating a line plot:
* creates a Plotly scatter trace representing a line plot
* **x_lines** x-coordinates of the points on the line with an empty default array
* **y_lines** y-coordinates of the points on the line with an empty default array

In [77]:
def line_plot(x_lines: np.array = np.array([]), y_lines: np.array = np.array([]), title: str = 'predicted function',
show_legend: bool = True, visible: bool = True, ) -> go.Scatter:

    scatter = go.Scatter(x=x_lines, y=y_lines, line=dict(color="orange", width=3), name=title, showlegend= show_legend, visible=visible)
    return scatter

define function for a scatter plot:
* creates a scatter plot with Plotly
* **x_dots** is an array of x-coordinates for the data points in the scatter plot with an empty default array
* **y_dots** is an array of y-coordinates for the data points in the scatter plot with an empty default array

In [78]:
def scatter_plot(visible: bool = True, x_dots: np.array = np.array([]), y_dots: np.array = np.array([]), name_dots: str = 'observed points',
show_legend: bool = True, color: str = 'red') -> go.Scatter:

    scatter = go.Scatter(x=x_dots, y=y_dots, visible=visible, mode='markers', name=name_dots, marker=dict(color=color, size=8), showlegend=show_legend)
    return scatter

define function for plotting the uncertainty area:
* generates an area plot to represent the uncertainty using the Plotly
* **x_lines** an array of x-coordinates for the lines outlining the area plot with an empty default array
* **y_lower** an array of y-coordinates representing the lower boundary of the uncertainty area plot with an empty default array
* **y_upper** an array of y-coordinates representing the upper boundary of the uncertainty area plot with an empty default array

In [79]:
def uncertainty_area_plot(visible: bool = True, x_lines: np.array = np.array([]), y_lower: np.array = np.array([]), y_upper: np.array = np.array([]), name: str = 'mean plus/minus standard deviation') -> go.Scatter:

    scatter = go.Scatter(x=np.concatenate((x_lines, x_lines[::-1])), y=np.concatenate((y_upper, y_lower[::-1])), visible=visible, fill='toself', fillcolor='rgba(189, 195, 199, 0.5)', line=dict(color='rgba(200, 200, 200, 0)'), hoverinfo='skip', showlegend=True, name=name)
    return scatter

define squared exponential kernel:
* **variable1** & **variable2** correspond to t & t'
* **sigma** describes the average distance away from the function mean and l determines the reach of influence on neighbors
* **sigma** & **length** is assigned the float standard value 1
* **variable_1** & **variable_2** expect an array

In [80]:
class SquaredExponentialKernel:
    def __init__(self, sigma: float = 1, length: float = 1, alpha: float = 0.75, beta: float = 0.75):
        self.sigma = sigma
        self.length = length
        self.alpha = alpha
        self.beta = beta

    def __call__(self, tau_1: np.array, tau_2: np.array):
        def kernel(variable_1, variable_2):
            k_SE = float(self.sigma**2 * np.exp(-(np.linalg.norm(variable_1 - variable_2)**2) /(2 * self.length**2)))
            return k_SE
        return kernel(tau_1, tau_2)

visualize the squared exponential kernel:
* only for testing the squared exponential kernel and monitor the effect of changing the parameters
* **@interact** generates a user interface which allows for an interactive regulation of the parameters length, sigma and variable_2 in the defined ranges
* the function **update** is called whenever the values of the sliders are changed
* **batch_update()** ensures that all updates to the figure within the block are batched together

In [81]:
x_lines = np.arange(-10, 10, 0.1)
kernel = SquaredExponentialKernel()  # initialize an instance of the class SquaredExponentialKernel, length and sigma have already standard values
data = line_plot(x_lines=x_lines, y_lines=np.array([kernel(x, 0) for x in x_lines]))  # generates the data for the line distribution

fig = go.FigureWidget(data)  # generates an interactive plot-widget with the passed data
fig = update_layout_of_graph(fig, title='squared exponential kernel')

@interact(length=(0.1, 3, 0.1), sigma=(0.1, 3, 0.1), variable_2=(-10, 10, 0.1))
def update(length=1, sigma=1, variable_2=0):
    with fig.batch_update():
        kernel = SquaredExponentialKernel(sigma=sigma, length=length)  # a new instance of the class SquaredExponentialKernel with the updated length and sigma value is generated
        fig.data[0].y = np.array([kernel(x, variable_2) for x in x_lines])  # updates the y-values plot data, data can contain multiply scatter plots
        # with data[0].y the y-coordinates of the first scatter plot are accessed

fig

interactive(children=(FloatSlider(value=1.0, description='length', max=3.0, min=0.1), FloatSlider(value=1.0, d…

FigureWidget({
    'data': [{'line': {'color': 'orange', 'width': 3},
              'name': 'predicted function',
              'showlegend': True,
              'type': 'scatter',
              'uid': '16034050-d732-4ef6-b685-c2d8ebd11263',
              'visible': True,
              'x': array([-1.00000000e+01, -9.90000000e+00, -9.80000000e+00, -9.70000000e+00,
                          -9.60000000e+00, -9.50000000e+00, -9.40000000e+00, -9.30000000e+00,
                          -9.20000000e+00, -9.10000000e+00, -9.00000000e+00, -8.90000000e+00,
                          -8.80000000e+00, -8.70000000e+00, -8.60000000e+00, -8.50000000e+00,
                          -8.40000000e+00, -8.30000000e+00, -8.20000000e+00, -8.10000000e+00,
                          -8.00000000e+00, -7.90000000e+00, -7.80000000e+00, -7.70000000e+00,
                          -7.60000000e+00, -7.50000000e+00, -7.40000000e+00, -7.30000000e+00,
                          -7.20000000e+00, -7.10000000e+00, -7.000000

functions for creating the covariance matrix, each of them were implemented separately in such a way as to make them comprehensible with the notes, which is why they are referenced:
* **val_1** & **val_2** are equal to a certain element of the arrays **t** & **t'**
* **alpha**, **R** & **C** are set to a specific value

In [82]:
# test

In [83]:
def k_fu(val_1, val_2, cov_func, R=0.87, C=330e-6, L=1e-3):
    alpha = cov_func.alpha
    beta = cov_func.beta
    l = cov_func.length

    c = R * C * cov_func(val_1, val_2)

    def integrand_a(tau):
        a = -1 / (gamma(1+alpha) * l**2)
        return a * (tau - val_2) * (val_1 - tau)**alpha * cov_func(tau, val_2)

    def integrand_b(tau):
        b = -1 / (gamma(1+alpha+beta) * l**2)
        return b * (tau - val_2) * (val_1 - tau)**(alpha+beta) * cov_func(tau, val_2)

    integral_val_a, _ = integrate(integrand_a, 0, val_1, epsabs=1e-9, epsrel=1e-9, limit=10000)
    integral_val_b, _ = integrate(integrand_b, 0, val_1, epsabs=1e-9, epsrel=1e-9, limit=10000)
    return integral_val_a + R / L * integral_val_b + c

In [84]:
def k_uf(val_1, val_2, cov_func, R=0.87, C=330e-6, L=1e-3):
    alpha = cov_func.alpha
    beta = cov_func.beta
    l = cov_func.length

    c = R * C * cov_func(val_1, val_2)

    def integrand_a(tau):
        a = 1 / (gamma(1+alpha) * l**2)
        return a * (val_1 - tau) * (val_2 - tau)**alpha * cov_func(val_1, tau)

    def integrand_b(tau):
        b = 1 / (gamma(1+alpha+beta) * l**2)
        return b * (val_1 - tau) * (val_2 - tau)**(alpha+beta) * cov_func(val_1, tau)

    integral_val_a, _ = integrate(integrand_a, 0, val_2, epsabs=1e-9, epsrel=1e-9, limit=10000)
    integral_val_b, _ = integrate(integrand_b, 0, val_2, epsabs=1e-9, epsrel=1e-9, limit=10000)
    return integral_val_a + R / L * integral_val_b + c

In [85]:
def k_uu(val_1, val_2, cov_func):
    return cov_func(val_1, val_2)

In [86]:
def k_ff(val_1, val_2, cov_func, R=0.87, C=330e-6, L=1e-3):
    alpha = cov_func.alpha
    beta = cov_func.beta
    l = cov_func.length

    # simple expressions, see notes:
    a = R * C * k_uf(val_1, val_2, cov_func, R, C, L)
    b = R * C * (k_fu(val_1, val_2, cov_func, R, C, L) - R * C * k_uu(val_1, val_2, cov_func))

    # more complex integral c, see notes:
    def c():
        def A(val_1, val_2, tau):
            a = 1/gamma(1+alpha) * (val_2 - tau)**alpha * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1/gamma(1 + alpha) * (val_1 - tau)**alpha * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        c = double_integral(val_1, val_2)
        return c

    # more complex integral d, see notes:
    def d():
        def A(val_1, val_2, tau):
            a = R / (L * gamma(1+alpha+beta)) * (val_2 - tau)**(alpha+beta) * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1/gamma(1 + alpha) * (val_1 - tau)**alpha * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        d = double_integral(val_1, val_2)
        return d

    # more complex integral e, see notes:
    def e():
        def A(val_1, val_2, tau):
            a = 1/gamma(1+alpha) * (val_2 - tau)**alpha * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1 / gamma(1 + alpha + beta) * (val_1 - tau)**(alpha + beta) * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        e = R / L * double_integral(val_1, val_2)
        return e

    # more complex integral f, see notes:
    def f():
        def A(val_1, val_2, tau):
            a = R / (L * gamma(1 + alpha + beta)) * (val_2 - tau)**(alpha + beta) * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1 / gamma(1 + alpha + beta) * (val_1 - tau)**(alpha + beta) * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        d = R / L * double_integral(val_1, val_2)
        return d

    return a + b + c() + d() + e() + f()

testing the implemented functions above:

In [87]:
val_1 = 0.1
val_2 = 0.1
x = np.arange(0.01, 3, 0.01)
cov_func = SquaredExponentialKernel()

for i in range(len(x)):
    kuf = k_uf(x[i], x[i], cov_func)
    kfu = k_fu(x[i], x[i], cov_func)
    kuu = k_uu(x[i], x[i], cov_func)
    kff = k_ff(x[i], x[i], cov_func)
    print('kuf = ', kuf, 'kuu = ', kuu, 'kfu = ', kfu, 'kff = ', kff)

kuf =  0.0003070494106657661 kuu =  1.0 kfu =  0.0003070494106657661 kff =  9.388136160428288e-08
kuf =  0.0005070424443475588 kuu =  1.0 kfu =  0.0005070424443475588 kff =  2.0871736154436826e-07
kuf =  0.001186970645058045 kuu =  1.0 kfu =  0.001186970645058045 kff =  5.991321343923294e-07
kuf =  0.002735931327708859 kuu =  1.0 kfu =  0.002735931327708859 kff =  1.4885453583704266e-06
kuf =  0.005613947511904068 kuu =  1.0 kfu =  0.005613947511904068 kff =  3.141102251335316e-06
kuf =  0.010341646737587954 kuu =  1.0 kfu =  0.010341646737587954 kff =  5.855747146723003e-06
kuf =  0.017493208442374654 kuu =  1.0 kfu =  0.017493208442374654 kff =  9.962173877611525e-06
kuf =  0.02769105925971123 kuu =  1.0 kfu =  0.02769105925971123 kff =  1.581777981692619e-05
kuf =  0.04160163276631616 kuu =  1.0 kfu =  0.04160163276631616 kff =  2.380523112441874e-05
kuf =  0.059931832236396795 kuu =  1.0 kfu =  0.059931832236396795 kff =  3.4330431660139035e-05
kuf =  0.08342598450086934 kuu =  1.0

a function for creating the sub and covariance matrix as described in the notes:
* works only correctly for squared matrices [n1 x n2] where n1 = n2

In [88]:
def create_sub_matrix(variable_1, variable_2, specified_cov_function, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    sub_matrix = np.zeros((n1, n2))

    for i, val_1 in enumerate(variable_1):
        for j, val_2 in enumerate(variable_2):
            sub_matrix[i, j] = specified_cov_function(val_1, val_2, cov_function)

    return sub_matrix

a function for creating the covariance matrix as described in the notes:
* works only correctly for squared matrices [2*n1 x 2*n2] where n1 = n2

In [89]:
def create_cov_matrix(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    cov = np.zeros((2*n1, 2*n2))

    kuu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uu, cov_function)
    kuf_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uf, cov_function)
    kfu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_fu, cov_function)
    kff_sub_matrix = create_sub_matrix(variable_1, variable_2, k_ff, cov_function)

    for i in range(2*n1):
        for j in range(2*n2):

            if i <= n1 - 1 and j <= n2 - 1:
                #print('kuu')
                cov[i, j] = kuu_sub_matrix[i, j]
            elif i <= n1 - 1 and n2 <= j <= 2*n2 - 1:
                #print('kuf')
                cov[i, j] = kuf_sub_matrix[i, j - n2]
            elif n1 <= i <= 2*n1 - 1 and j <= n2 - 1:
                #print('kfu')
                cov[i, j] = kfu_sub_matrix[i - n1, j]
            else:
                #print('kff')
                cov[i, j] = kff_sub_matrix[i - n1, j - n2]

    return cov

testing the implemented functions for creating the covariance matrix above

In [90]:
variable_1 = [1, 1, 1, 1]
variable_2 = [1, 1, 1, 1]
#variable_1 = np.array([0.3, 0.8, 1.4, 1.8, 2])
#variable_2 = np.array([0.3, 0.8, 1.4, 1.8, 2])

cov_func = SquaredExponentialKernel()
cov = create_cov_matrix(variable_1, variable_2, cov_func)

print('covariance matrix = ', cov)

covariance matrix =  [[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02]
 [1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02
  7.88951653e-02 7.88951653e-02 7.88951653e-02 7.88951653e-02]
 [1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02
  7.88951653e-02 7.88951653e-02 7.88951653e-02 7.88951653e-02]
 [1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02
  7.88951653e-02 7.88951653e-02 7.88951653e-02 7.88951653e-02]
 [1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02
  7.88951653e-02 7.88951653e-02 7.88951653

define function for calculating q_u transposed


In [91]:
def q_u_transposed(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    q_u_trans = np.zeros((n1, 2*n2))

    kuu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uu, cov_function)
    kuf_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uf, cov_function)

    for i in range(n1):
        for j in range(2*n2):

            if j <= n2 - 1:
                q_u_trans[i, j] = kuu_sub_matrix[i, j]
            else:
                q_u_trans[i, j] = kuf_sub_matrix[i, j - n2]

    return q_u_trans

In [92]:
def q_f_transposed(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    q_f_trans = np.zeros((n1, 2*n2))

    kfu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_fu, cov_function)
    kff_sub_matrix = create_sub_matrix(variable_1, variable_2, k_ff, cov_function)

    for i in range(n1):
        for j in range(2*n2):

            if j <= n2 - 1:
                q_f_trans[i, j] = kfu_sub_matrix[i, j]
            else:
                q_f_trans[i, j] = kff_sub_matrix[i, j - n2]

    return q_f_trans

testing if the function q_u_transposed works properly

In [93]:
variable_1 = [1, 1, 1, 1]
variable_2 = [1, 2, 1, 1]
cov_func = SquaredExponentialKernel()

q_u_trans = q_u_transposed(variable_1, variable_2, cov_func)
print('q_u transposed = ', q_u_trans)

q_u transposed =  [[  1.           0.60653066   1.           1.         137.40029216
  475.66522877 137.40029216 137.40029216]
 [  1.           0.60653066   1.           1.         137.40029216
  475.66522877 137.40029216 137.40029216]
 [  1.           0.60653066   1.           1.         137.40029216
  475.66522877 137.40029216 137.40029216]
 [  1.           0.60653066   1.           1.         137.40029216
  475.66522877 137.40029216 137.40029216]]


new training method: page 113 in GP for ML

In [94]:
def log_marginal_likelihood(data_x, data_y, cov_func, noise):
    n = len(data_x)
    y = np.concatenate([data_y, data_y])
    y_transposed = y.T
    K = create_cov_matrix(data_x, data_x, cov_func) + noise * np.identity(2*n)
    K_inverse = np.linalg.inv(K)

    a = n/2 * np.log(2*np.pi)  # normalization constant
    b = 1/2 * np.log(np.linalg.norm(K))  # complexity penalty
    c = 1/2 * np.dot(y, np.dot(K_inverse, y_transposed))  # involving observed targets
    log_ml = c + b + a  # is actually the negative log marginal likelihood
    return log_ml

sigma_vals = np.arange(0.2, 0.4, 0.05)
length_vals = np.arange(1, 1.9, 0.05)
memory = {'sigma': 1, 'length': 1, 'negative_log_marginal_likelihood': 1000}

data_x = np.array([0.04507640906154828, 0.2505406250747911, 0.6104736607890773, 1.214547321503358, 2.00835312507477, 2.711478125074755, 3.302298437574742])
data_y = np.array([-1.781453, -0.9913542, -0.2168788, 0.1644358, 0.1259032, 0.05078084, 0.01758787])

for sigma_index, sigma_val in enumerate(sigma_vals):
    for l_index, length_val in enumerate(length_vals):
        cov_func = SquaredExponentialKernel(sigma=sigma_val, length=length_val)
        negative_log_marginal_likelihood = log_marginal_likelihood(data_x, data_y, cov_func, noise = 0.01)
        print('likelihood = ', negative_log_marginal_likelihood)

        if negative_log_marginal_likelihood < memory['negative_log_marginal_likelihood']:
            memory['length'] = length_val
            memory['sigma'] = sigma_val
            memory['negative_log_marginal_likelihood'] = negative_log_marginal_likelihood

        print('obtained sigma and l values that minimize the negative log marginal likelihood: ', 'sigma = ', memory['sigma'], 'length = ', memory['length'])

likelihood =  222.85405383689468
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  0.2 length =  1.0


KeyboardInterrupt: 

In [ ]:
# please
cov_func = SquaredExponentialKernel(sigma=0.3, length=1.45)
negative_log_marginal_likelihood = log_marginal_likelihood(data_x, data_y, cov_func, noise = 0.01)
print(negative_log_marginal_likelihood)

define class GPR:
* **data_x** is the input of the observed points
* **data_y** is the output of the observed points
* **noise** is the noise in the observed data
* **memory** is a variable where the mean, the covariance matrix and the variance is stored
* **3e-7** add this value to the noise in order to ensure stability when inverting the covariance matrix

In [95]:
class GPR:
    def __init__(self, data_x, data_y, cov_func= SquaredExponentialKernel(sigma=memory['sigma'], length=memory['length']), noise=0.01):
        print('sigma memory = ', memory['sigma'])
        print('length memory = ', memory['length'])
        self.data_x = data_x
        self.data_y = np.concatenate([data_y, data_y])
        self.cov_func = cov_func
        self.noise = noise
        self.memory = None

        n = len(data_x)
        self.inv_cov_matrix_of_input_data = np.linalg.inv(create_cov_matrix(data_x, data_x, cov_func) + (noise + 3e-7)**2 * np.identity(2*n))

    def predict(self, at_values):
        q_u_trans = q_u_transposed(at_values, self.data_x, self.cov_func)
        q_f_trans = q_f_transposed(at_values, self.data_x, self.cov_func)

        # methode paper:
        mean_at_values_u = np.dot(q_u_trans, np.dot(self.data_y, self.inv_cov_matrix_of_input_data.T).T).flatten()
        #mean_at_values_f = np.dot(q_f_trans, np.dot(self.data_y, self.inv_cov_matrix_of_input_data.T).T).flatten()

        covariance_matrix_u = create_sub_matrix(at_values, at_values, k_uu, self.cov_func) - np.dot(q_u_trans, np.dot(self.inv_cov_matrix_of_input_data, q_u_trans.T))
        #covariance_matrix_f = create_sub_matrix(at_values, at_values, k_ff, self.cov_func) - np.dot(q_f_trans, np.dot(self.inv_cov_matrix_of_input_data, q_f_trans.T))

        variance_u = np.abs(np.diag(covariance_matrix_u))
        #variance_f = np.abs(np.diag(covariance_matrix_f))

        self.memory = {'mean': mean_at_values_u, 'covariance_matrix':covariance_matrix_u, 'variance': variance_u}

        return mean_at_values_u

defining function for plotting the GPR:
* **data_x** is a list of observed input data points
* **data_y** is a list of observed output data points
* **x** is equivalent to **at_values** in the GPR, the values we want to do predictions on

In [ ]:
def plot_GPR(data_x, data_y, model, x, data_x_mv, data_y_mv, visible=True) -> list:
    mean = model.predict(x)
    standard_deviation = np.sqrt(model.memory['variance'])
    data = []

    for i in range(1, 4):
        data.append(uncertainty_area_plot(x_lines=x, y_lower=mean -i * standard_deviation, y_upper=mean + i*standard_deviation, name=f'mean plus/minus {i}*standard deviation', visible=visible))

    data.append(scatter_plot(x_dots=data_x_mv, y_dots=data_y_mv, visible=visible, color='blue'))
    data.append(scatter_plot(x_dots=data_x, y_dots=data_y, visible=visible, color='red'))
    data.append(line_plot(x_lines=x, y_lines=mean, visible=visible))
    return data

initializing the training data

In [ ]:
x_values = np.array([0.04507640906154828, 0.6104736607890773, 1.214547321503358, 2.00835312507477, 2.711478125074755])
y_values = np.array([-1.781453, -0.2168788, 0.1644358, 0.1259032, 0.05078084])

x_values_mv = np.array([9.765625114122045e-08, 0.02253825335889971, 0.04507640906154828, 0.06761456476419686, 0.09015272046684542, 0.11269087616949401, 0.1352290318721426, 0.1577671875747911, 0.17241562507479108, 0.1870640625747911, 0.1968296875747911, 0.20659531257479108, 0.2114781250747911, 0.2163609375747911, 0.2212437500747911, 0.2261265625747911, 0.23100937507479108, 0.24077500007479122, 0.2505406250747911, 0.2651890625747911, 0.28820803578907694, 0.3112270090033626, 0.3342459822176484, 0.3572649554319341, 0.3802839286462198, 0.4033029018605056, 0.4263218750747913, 0.4570138393605056, 0.4877058036462199, 0.5183977679319343, 0.5490897322176486, 0.579781696503363, 0.6104736607890773, 0.6411656250747915, 0.6613944197176487, 0.6816232143605058, 0.701852009003363, 0.7220808036462202, 0.7423095982890774, 0.7625383929319345, 0.7827671875747916, 0.7925328125747916, 0.8022984375747916, 0.8071812500747916, 0.8120640625747916, 0.8169468750747917, 0.8218296875747917, 0.8267125000747917, 0.8315953125747917, 0.8413609375747917, 0.8560093750747917, 0.8706578125747917, 0.8943743304319344, 0.918090848289077, 0.9418073661462198, 0.9655238840033625, 0.9892404018605051, 1.012956919717648, 1.036673437574791, 1.072248214360504, 1.1078229911462179, 1.143397767931931, 1.1789725447176451, 1.214547321503358, 1.250122098289072, 1.285696875074785, 1.331734821503356, 1.3777727679319258, 1.423810714360497, 1.469848660789067, 1.515886607217637, 1.561924553646208, 1.607962500074778, 1.6651611607890628, 1.722359821503347, 1.779558482217632, 1.836757142931917, 1.893955803646201, 1.951154464360486, 2.00835312507477, 2.0634591518604832, 2.1185651786461963, 2.17367120543191, 2.228777232217623, 2.283883259003336, 2.338989285789049, 2.3940953125747617, 2.4394357143604752, 2.484776116146188, 2.530116517931902, 2.575456919717615, 2.620797321503329, 2.666137723289042, 2.711478125074755, 2.747750446503326, 2.784022767931896, 2.820295089360467, 2.856567410789038, 2.892839732217609, 2.929112053646179, 2.96538437507475, 2.992588616146178, 3.0197928572176056, 3.046997098289034, 3.074201339360462, 3.10140558043189, 3.1286098215033182, 3.155814062574746, 3.171160044717603, 3.1865060268604593, 3.201852009003316, 3.217197991146173, 3.23254397328903, 3.247889955431887, 3.263235937574743, 3.268118750074743, 3.273001562574743, 3.277884375074743, 3.282767187574743, 3.287650000074743, 3.292532812574743, 3.302298437574742, 3.316946875074742, 3.3315953125747417, 3.355311830431884, 3.379028348289027, 3.4027448661461692, 3.426461384003311, 3.450177901860454, 3.4738944197175963, 3.4976109375747377, 3.527605357217595, 3.557599776860451, 3.587594196503308, 3.6175886161461643, 3.6475830357890207, 3.677577455431878, 3.7075718750747337, 3.727103125074733, 3.746634375074733, 3.766165625074733, 3.785696875074732, 3.805228125074732, 3.824759375074731, 3.844290625074731, 3.854056250074731, 3.86382187507473, 3.8687046875747297, 3.8735875000747297, 3.87847031257473, 3.88335312507473, 3.8882359375747297, 3.8931187500747297, 3.90288437507473, 3.9175328125747293, 3.932181250074729, 3.9558977679318716, 3.9796142857890136, 4.003330803646156])

y_values_mv = np.array([-2.0, -1.888619, -1.781453, -1.678503, -1.579768, -1.485249, -1.394945, -1.308856, -1.254713, -1.202122, -1.167907, -1.134358, -1.117829, -1.101464, -1.085259, -1.069215, -1.053329, -1.022031, -0.9913542, -0.9464839, -0.8789595, -0.8143604, -0.7526864, -0.6939376, -0.6381139, -0.5852155, -0.5352421, -0.4727319, -0.4140015, -0.359051, -0.3078804, -0.2604897, -0.2168788, -0.1770478, -0.1522851, -0.1287013, -0.1062962, -0.08507001, -0.06502261, -0.04615404, -0.02846429, -0.02023892, -0.0122477, -0.008338474, -0.004486084, -0.0006899731, 0.003050414, 0.006735627, 0.01036621, 0.01746566, 0.02771855, 0.03750848, 0.05228297, 0.06606884, 0.07886609, 0.09067473, 0.1014948, 0.1113262, 0.120169, 0.1317869, 0.1420226, 0.150876, 0.158347, 0.1644358, 0.1691423, 0.1724666, 0.1752863, 0.1770421, 0.1777339, 0.1773618, 0.1759258, 0.1734258, 0.1698619, 0.1647158, 0.1591918, 0.1532898, 0.14701, 0.1403523, 0.1333167, 0.1259032, 0.1189408, 0.1121284, 0.105466, 0.09895367, 0.09259135, 0.08637904, 0.08031677, 0.07556673, 0.07099357, 0.06659727, 0.06237786, 0.05833531, 0.05446964, 0.05078084, 0.04796508, 0.04525736, 0.0426577, 0.04016609, 0.03778253, 0.03550702, 0.03333956, 0.03177897, 0.03027079, 0.02881502, 0.02741165, 0.0260607, 0.02476215, 0.02351601, 0.0228333, 0.02216514, 0.02151156, 0.02087254, 0.0202481, 0.01963821, 0.0190429, 0.01885621, 0.01867091, 0.01848699, 0.01830444, 0.01812327, 0.01794345, 0.01758787, 0.01706454, 0.01655306, 0.0157516, 0.01497798, 0.0142322, 0.01351427, 0.01282419, 0.01216194, 0.01152755, 0.01076226, 0.01003217, 0.009337263, 0.008677548, 0.008053023, 0.007463688, 0.006909542, 0.006563731, 0.006229627, 0.00590723, 0.005596541, 0.005297558, 0.005010284, 0.004734716, 0.004600502, 0.00446889, 0.004404049, 0.004339844, 0.004276272, 0.004213326, 0.004151003, 0.004089296, 0.003967719, 0.003789871, 0.003617334, 0.003350114, 0.003094816, 0.002851439])

print(len(x_values_mv), len(y_values_mv))

x = np.arange(0.01, 4, 0.01)

plot the output of the GPR

In [ ]:
model = GPR(x_values, y_values)
data = plot_GPR(data_x=x_values, data_y=y_values, x=x, model=model, data_x_mv=x_values_mv, data_y_mv=y_values_mv)
fig4 = go.Figure(data)
fig4 = update_layout_of_graph(fig=fig4, title=f'GPR with length {model.cov_func.length}, sigma {model.cov_func.sigma}, alpha {cov_func.alpha}, beta {model.cov_func.beta} and noise {model.noise}')
fig4.show()
fig4.write_html('C:\\Users\\simon\\Desktop\\RLC_Current\\RCL_Current_aperiodic_0.75.html')